# Lab02: Frequent itemset mining

- Student ID: 20127068
- Student name: Hồ Minh Thanh Tài

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [5]:
from collections import defaultdict

### Read data
First we have to read data from database

In [6]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [7]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = list(set(a + b))
    
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        # support = 0
        # temp = {}
        # compare_num = len(itemset)

        # for tid in range(len(self.data)):
        #     temp = set(itemset).intersection(self.data[tid + 1])
        #     if (compare_num == len(temp)):
        #         support += 1
        #     temp.clear()

        support = 0
        for transaction in self.data.values():
            if set(itemset).issubset(transaction):
                support += 1

        return support

    # This function has a litle confusing so I have to fix it get better output
    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 1:
            return search_space[itter_node]
        subtree = search_space[itter_node]
        for key in subtree.keys():
            if key in ['itemset','pruned','support']: continue
            return self.get_sub_tree(k - 1, tree, search_space, key)
          


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO

        for key in search_space.keys():
            if key in ['itemset','pruned','support']: continue
            if search_space[key]['support'] < self.minSup:
                search_space[key]['pruned'] = True
            else:
                self.L[k].append(search_space[key]['itemset'])
        
        
    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]            
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])

                # Second add newset to search_space
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['support'] = self.computeItemsetSupport(newset)
                if search_space[a][b]['support'] < self.minSup:
                    search_space[a][b]['pruned'] = True
                else:
                    search_space[a][b]['pruned'] = False
            
            
            sub_tree = tree[a]
            sub_search_space = search_space[a]
                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [8]:
data, s= readData('chess.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'chess.txt'

In [ ]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [40, 34], [34, 29], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [60, 7], [40, 7], [29, 7], [52, 7], [58, 7], [36, 60], [40, 36], [36, 29], [36, 52], [58, 36], [40, 60], [60, 29], [60, 52], [58, 60], [40, 29], [40, 52], [40, 58], [52, 29], [58, 29], [58, 52]], 3: [[48, 58, 52], [56, 52, 29], [56, 58, 29], [56, 58, 52], [66, 60, 29], [66, 60, 52], [66, 60, 58], [66, 52, 29], [66, 58, 29], [66, 52, 58], [40, 34, 29], [40, 34, 52], [40, 34, 58], [34, 52, 29], [34, 58, 29], [34, 52, 58], [60, 29, 62], [60, 62, 52], [58, 60, 62], [40, 29, 62], [40, 52, 62], [40, 58, 62], [52, 29, 62], [58, 29, 62], [58, 52, 62], [40, 60, 7], [60, 29, 7], [60, 52, 7], [58, 60, 7], [40, 29, 7], [40, 52, 7], [40, 58, 7], [52, 29, 7], [58, 29, 7], [58, 52, 7], [40, 36, 60], [36, 29, 60], [36, 60, 52], [58, 36, 60], [40

### Answer questions here:
**Why don't we compute support of items while reading data?**<br>
Because the result must be calculated after searching all the data, which is impossible when reading the data while the data is incomplete.

**why should we do sort**<br>
It would be simple to skip all of the candidates who did not meet the minimum support requirement for the sorted set of candidates.
In other words, all items with an occurrence of less than minSup value would be eliminated out, leaving only the reasonable items.

**study about python set and its advantages ?**<br>
The benefit of using Set in Python is that all of the values are duplicate are not allowed, saving us time from having to verify several instances of the same support or itemset with identical values.<br>
Secondly, Set in python provides a lot of methods to handle the calculation between two sets such as <b>intersect</b>, <b>issubset</b>, etc.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**<br>
Due to the fact that all operations still depend on loops and recursion loops in the practical implementation, deleting all keys from the search_space and tree at once is impossible.<br>
The loops may encounter an issue if a key is removed since the indices were altered during operation.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**<br>
Once the frequent itemset is found using the Apriori technique, all of its subsets must likewise be frequent. Candidate itemset are generated by the Apriori algorithm, which then determines their frequency. Because the algorithm must verify every value in every list to determine whether the item occurred or not, its complexity could be O(n).<br>
The Tree Projection technique mines the frequent patterns from databases via pattern fragment growth. Since the logic in the Tree basic algorithm moved brands one at a time, the complexity may be O(log2(n)).


# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

FILE_FORMAT = 'html.parser'
FILE_NAME = "churn.txt"
INFOR_TAG = 'td'

## 1. Firstly, we preprocessing file

In [ ]:
def split_text(text):
    return text[text.index('>')+1 : text.rfind('<')]

with open(FILE_NAME) as open_file:
    soup = BeautifulSoup(open_file, FILE_FORMAT)

found = soup.find_all(INFOR_TAG)
features = split_text(str(found[1])).split(',')

### Problem: Find out the itemsets of features that contribute the most on user's loyalty. 

## 2. Data Frame Generation

In [ ]:
df = pd.DataFrame(columns=features)

for i in range(3, len(found), 2):
    string = str(found[i])
    values = split_text(string)
    dic_val = defaultdict(lambda: 0)
    for key, val in zip(features, values.split(',')):
        dic_val[key] = val
    df = pd.concat([df, pd.DataFrame([dic_val])], ignore_index=True)
df.head(5)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.100000,110,45.070000,...,99,16.780000,244.700000,91,11.010000,10.000000,3,2.700000,1,False.
1,OH,107,415,371-7191,no,yes,26,161.600000,123,27.470000,...,103,16.620000,254.400000,103,11.450000,13.700000,3,3.700000,1,False.
2,NJ,137,415,358-1921,no,no,0,243.400000,114,41.380000,...,110,10.300000,162.600000,104,7.320000,12.200000,5,3.290000,0,False.
3,OH,84,408,375-9999,yes,no,0,299.400000,71,50.900000,...,88,5.260000,196.900000,89,8.860000,6.600000,7,1.780000,2,False.
4,OK,75,415,330-6626,yes,no,0,166.700000,113,28.340000,...,122,12.610000,186.900000,121,8.410000,10.100000,3,2.730000,3,False.


### I will ignore all the categorical columns and just focus only on numerical columns for extracting solution.

In [ ]:
num_features = []
df = df.loc[df['Churn?']=='False.']

for feature in features:
    try:
        df[feature] = df[feature].astype(float)
        num_features.append(feature)
    except:
        df = df.drop(feature, axis=1)
df.head(5)

,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,128.0,415.0,25.0,265.1,110.0,45.07,197.4,99.0,16.78,244.7,91.0,11.01,10.0,3.0,2.70,1.0
1,107.0,415.0,26.0,161.6,123.0,27.47,195.5,103.0,16.62,254.4,103.0,11.45,13.7,3.0,3.70,1.0
2,137.0,415.0,0.0,243.4,114.0,41.38,121.2,110.0,10.30,162.6,104.0,7.32,12.2,5.0,3.29,0.0
3,84.0,408.0,0.0,299.4,71.0,50.90,61.9,88.0,5.26,196.9,89.0,8.86,6.6,7.0,1.78,2.0
4,75.0,415.0,0.0,166.7,113.0,28.34,148.3,122.0,12.61,186.9,121.0,8.41,10.1,3.0,2.73,3.0


## 3. Z-Score normalizing all the values and then compare with limit to define into binary.

In [ ]:
for i in range(len(num_features)):
    col_mean = df[num_features[i]].mean()
    n = df[num_features[i]].shape[0]
    df[num_features[i]] = df[num_features[i]].apply(lambda x: (x-col_mean)/(np.abs(x-col_mean).sum()/n))
    # z-score normalizing process

limit = 0
for feature in num_features:
    df.loc[df[feature]>limit, feature] = 1
    df.loc[df[feature]<=limit, feature] = 0

df.head(5)

,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


Wherever containing 1.0 mean this feature is one of itemset in current transaction.

In [ ]:
data = {}
s = defaultdict(lambda: 0)

for index, row in df.iterrows():
    itemset = [feature for feature in num_features if row[feature]==1.0]
    for item in itemset:
        s[item]+=1
    data[index] = set(itemset)

for key in data.keys():
    print(key, data[key])

0 {'Day Mins', 'Day Calls', 'Account Length', 'Night Mins', 'Day Charge', 'Night Charge', 'VMail Message'}
1 {'Night Calls', 'Day Calls', 'Intl Mins', 'Eve Calls', 'Night Mins', 'Account Length', 'Intl Charge', 'Night Charge', 'VMail Message'}
2 {'Night Calls', 'Day Mins', 'Day Calls', 'Intl Mins', 'Eve Calls', 'Intl Charge', 'Account Length', 'Intl Calls', 'Day Charge'}
3 {'Intl Calls', 'Day Charge', 'Day Mins', 'CustServ Calls'}
4 {'Night Calls', 'Eve Calls', 'CustServ Calls', 'Day Calls'}
5 {'Night Charge', 'Night Calls', 'Area Code', 'Day Mins', 'Eve Calls', 'Account Length', 'Night Mins', 'Eve Mins', 'Day Charge', 'Eve Charge', 'Intl Calls'}
6 {'Night Charge', 'Night Calls', 'Area Code', 'Day Mins', 'Eve Calls', 'Account Length', 'Night Mins', 'Eve Mins', 'Day Charge', 'Eve Charge', 'Intl Calls', 'CustServ Calls', 'VMail Message'}
7 {'Intl Calls', 'Night Charge', 'Night Mins', 'Account Length'}
8 {'Night Charge', 'Day Mins', 'Account Length', 'Night Mins', 'Eve Mins', 'Day Charge'

## 4. TreeProjection with minSup=30%

In [ ]:
tp = TP(data, s, minSup=round(len(data)*0.3, 0))
result = tp.miningResults()

### Outcome

In [ ]:
for key in result.keys():
    print(key, result[key])

1 [['CustServ Calls'], ['Intl Calls'], ['Night Calls'], ['Eve Calls'], ['Account Length'], ['Day Calls'], ['Night Mins'], ['Night Charge'], ['Eve Mins'], ['Eve Charge'], ['Intl Mins'], ['Intl Charge'], ['Day Charge'], ['Day Mins']]
2 [['Night Charge', 'Night Mins'], ['Eve Mins', 'Eve Charge'], ['Intl Mins', 'Intl Charge'], ['Day Charge', 'Day Mins']]
3 []


=> There are 17 itemsets that satisfied the condition of loyal customers with the min support of 30%

# 4 References

Feel free to send questions to my email address: ntthuhang0131@gmail.com
